In [ ]:
import numpy as np
import os
import io
import cv2

import torch

import sys
PATH_ROOT=os.path.realpath(os.path.join(os.curdir, '..'))
sys.path.append(PATH_ROOT)
print(PATH_ROOT)


In [ ]:
for m in ['demo_config', 'demo_utils']:
    if m in sys.modules:
        del sys.modules[m]

In [ ]:

from demo_config import (Config,
                    eval_dict_leaf)

from demo_utils import (retrieve_text,
                  _frame_from_video,
                  setup_internvideo2)


In [ ]:
video = cv2.VideoCapture('example1.mp4')
frames = [x for x in _frame_from_video(video)]

In [ ]:
text_candidates = ["A playful dog and its owner wrestle in the snowy yard, chasing each other with joyous abandon.",
                   "A man in a gray coat walks through the snowy landscape, pulling a sleigh loaded with toys.",
                   "A person dressed in a blue jacket shovels the snow-covered pavement outside their house.",
                   "A pet dog excitedly runs through the snowy yard, chasing a toy thrown by its owner.",
                   "A person stands on the snowy floor, pushing a sled loaded with blankets, preparing for a fun-filled ride.",
                   "A man in a gray hat and coat walks through the snowy yard, carefully navigating around the trees.",
                   "A playful dog slides down a snowy hill, wagging its tail with delight.",
                   "A person in a blue jacket walks their pet on a leash, enjoying a peaceful winter walk among the trees.",
                   "A man in a gray sweater plays fetch with his dog in the snowy yard, throwing a toy and watching it run.",
                   "A person bundled up in a blanket walks through the snowy landscape, enjoying the serene winter scenery."]

In [ ]:
config = Config.from_file('internvideo2_stage2_config.py')
config = eval_dict_leaf(config)

In [ ]:
model_pth = '/cwd/weights/pretrained/InternVideo2-stage2_1b-224p-f4.pt'
config['pretrained_path'] = model_pth
config['model']['vision_encoder']['pretrained']='/cwd/weights/pretrained/InternVideo2-stage2_1b-224p-f4.pt'
config['model']['text_encoder']['config']='../configs/config_bert_large.json'

In [ ]:
intern_model, tokenizer = setup_internvideo2(config)

In [ ]:
intern_model.eval()
intern_model.cuda()
print(1)

In [ ]:
texts, probs = retrieve_text(frames, text_candidates, model=intern_model, topk=5, config=config)

for t, p in zip(texts, probs):
    print(f'text: {t} ~ prob: {p:.4f}')

In [ ]:
import torch_tensorrt

import torch
from torch import nn

In [ ]:
class VideoEncoder(nn.Module):
    def __init__(self, vlm: nn.Module):
        super().__init__()
        self.vlm = vlm

    def forward(self, frames: torch.Tensor):
        _, vfeat = self.vlm.encode_vision(frames, test=True)
        vfeat = self.vlm.vision_proj(vfeat)
        vfeat /= vfeat.vlm.norm(dim=-1, keepdim=True)
        return vfeat

video_encoder = VideoEncoder(intern_model)

trt_model = torch_tensorrt.compile(video_encoder,
    inputs= [torch_tensorrt.Input((1, 4, 3, 224, 224))],
    enabled_precisions= { torch.half} # Run with FP32
)